In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from env import github_token, github_username
import unicodedata
from wrangle import wrangle, acquire
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
df = wrangle()

In [4]:
df.head()

,title,readme,lemmatized
0,freecodecamp,freecodecamp.org's open-source codebase and cu...,freecodecamporgs opensource codebase curriculu...
1,free-programming-books,list of free learning resources in many langua...,list free learning resource many language intr...
2,awesome,one task contents platforms programming langua...,one task content platform programming language...
3,coding-interview-university,coding interview university what is it? table ...,coding interview university table content stud...
4,996.icu,996.icu related press coverage: what is 996? w...,996icu related press coverage 996 voice state ...


In [6]:
df = acquire()
df.head()

,title,readme
0,freecodecamp,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nfreecodecamp.o...
1,free-programming-books,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nlist of free l...
2,awesome,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\none task\ncont...
3,coding-interview-university,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ncoding intervi...
4,996.icu,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n996.icu\nrelat...


In [2]:
url = 'https://github.com/EvanLi/Github-Ranking/blob/master/Top100/Top-100-stars.md'

In [3]:
headers={"Authorization": f"token {github_token}", "User-Agent": github_username}

In [4]:
response = requests.get(url, headers)

In [5]:
response

<Response [200]>

In [6]:
response.text[:200]

'{"payload":{"allShortcutsEnabled":false,"fileTree":{"Top100":{"items":[{"name":"ActionScript.md","path":"Top100/ActionScript.md","contentType":"file"},{"name":"C.md","path":"Top100/C.md","contentType"'

In [7]:
response.json

<bound method Response.json of <Response [200]>>

In [8]:
soup = BeautifulSoup(response.text, 'html.parser')

In [8]:
soup

{"payload":{"allShortcutsEnabled":false,"fileTree":{"Top100":{"items":[{"name":"ActionScript.md","path":"Top100/ActionScript.md","contentType":"file"},{"name":"C.md","path":"Top100/C.md","contentType":"file"},{"name":"CPP.md","path":"Top100/CPP.md","contentType":"file"},{"name":"CSS.md","path":"Top100/CSS.md","contentType":"file"},{"name":"CSharp.md","path":"Top100/CSharp.md","contentType":"file"},{"name":"Clojure.md","path":"Top100/Clojure.md","contentType":"file"},{"name":"CoffeeScript.md","path":"Top100/CoffeeScript.md","contentType":"file"},{"name":"DM.md","path":"Top100/DM.md","contentType":"file"},{"name":"Dart.md","path":"Top100/Dart.md","contentType":"file"},{"name":"Elixir.md","path":"Top100/Elixir.md","contentType":"file"},{"name":"Go.md","path":"Top100/Go.md","contentType":"file"},{"name":"Groovy.md","path":"Top100/Groovy.md","contentType":"file"},{"name":"HTML.md","path":"Top100/HTML.md","contentType":"file"},{"name":"Haskell.md","path":"Top100/Haskell.md","contentType":"fi

In [15]:
repo_links = soup.find_all('a')

In [16]:
repos = [link_tag['href'] for link_tag in repo_links][3:]

In [11]:
repos[0][2:-2]

'https://github.com/freeCodeCamp/freeCodeCamp'

In [12]:
i = 0 
for repo in repos:
    repos[i] = repos[i][2:-2]
    i += 1
repos

['https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/EbookFoundation/free-programming-books',
 'https://github.com/sindresorhus/awesome',
 'https://github.com/jwasham/coding-interview-university',
 'https://github.com/996icu/996.ICU',
 'https://github.com/public-apis/public-apis',
 'https://github.com/kamranahmedse/developer-roadmap',
 'https://github.com/donnemartin/system-design-primer',
 'https://github.com/codecrafters-io/build-your-own-x',
 'https://github.com/facebook/react',
 'https://github.com/vuejs/vue',
 'https://github.com/vuejs/core',
 'https://github.com/vinta/awesome-python',
 'https://github.com/tensorflow/tensorflow',
 'https://github.com/trekhleb/javascript-algorithms',
 'https://github.com/getify/You-Dont-Know-JS',
 'https://github.com/TheAlgorithms/Python',
 'https://github.com/CyC2018/CS-Notes',
 'https://github.com/twbs/bootstrap',
 'https://github.com/ohmyzsh/ohmyzsh',
 'https://github.com/torvalds/linux',
 'https://github.com/flutter/flutter',
 

In [13]:
url2 = 'https://github.com/trending?since=daily'
headers2={"Authorization": f"token {github_token}", "User-Agent": github_username}
response2 = requests.get(url2, headers2)
soup2 = BeautifulSoup(response2.text, 'html.parser')
repo_links2 = soup2.find_all('a')
repos2 = [link_tag['href'] for link_tag in repo_links2]

In [14]:
link_one = f'https://github.com{repos2[948]}'
link_two = f'https://github.com{repos2[957]}'
link_three = f'https://github.com{repos2[966]}'
link_four = f'https://github.com{repos2[975]}'
link_five = f'https://github.com{repos2[984]}'

In [15]:
repos

['https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/EbookFoundation/free-programming-books',
 'https://github.com/sindresorhus/awesome',
 'https://github.com/jwasham/coding-interview-university',
 'https://github.com/996icu/996.ICU',
 'https://github.com/public-apis/public-apis',
 'https://github.com/kamranahmedse/developer-roadmap',
 'https://github.com/donnemartin/system-design-primer',
 'https://github.com/codecrafters-io/build-your-own-x',
 'https://github.com/facebook/react',
 'https://github.com/vuejs/vue',
 'https://github.com/vuejs/core',
 'https://github.com/vinta/awesome-python',
 'https://github.com/tensorflow/tensorflow',
 'https://github.com/trekhleb/javascript-algorithms',
 'https://github.com/getify/You-Dont-Know-JS',
 'https://github.com/TheAlgorithms/Python',
 'https://github.com/CyC2018/CS-Notes',
 'https://github.com/twbs/bootstrap',
 'https://github.com/ohmyzsh/ohmyzsh',
 'https://github.com/torvalds/linux',
 'https://github.com/flutter/flutter',
 

In [16]:
repos.append(link_one)
repos.append(link_two)
repos.append(link_three)
repos.append(link_four)
repos.append(link_five)

In [17]:
repos

['https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/EbookFoundation/free-programming-books',
 'https://github.com/sindresorhus/awesome',
 'https://github.com/jwasham/coding-interview-university',
 'https://github.com/996icu/996.ICU',
 'https://github.com/public-apis/public-apis',
 'https://github.com/kamranahmedse/developer-roadmap',
 'https://github.com/donnemartin/system-design-primer',
 'https://github.com/codecrafters-io/build-your-own-x',
 'https://github.com/facebook/react',
 'https://github.com/vuejs/vue',
 'https://github.com/vuejs/core',
 'https://github.com/vinta/awesome-python',
 'https://github.com/tensorflow/tensorflow',
 'https://github.com/trekhleb/javascript-algorithms',
 'https://github.com/getify/You-Dont-Know-JS',
 'https://github.com/TheAlgorithms/Python',
 'https://github.com/CyC2018/CS-Notes',
 'https://github.com/twbs/bootstrap',
 'https://github.com/ohmyzsh/ohmyzsh',
 'https://github.com/torvalds/linux',
 'https://github.com/flutter/flutter',
 

In [18]:
len(repos)

107

In [19]:
url = repos[0]
response = requests.get(url, headers={"Authorization": f"token {github_token}", "User-Agent": github_username})
soup = BeautifulSoup(response.text, 'html.parser')
soup


<!DOCTYPE html>

<html data-a11y-animated-images="system" data-a11y-link-underlines="true" data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
<link href="https://avatars.githubusercontent.com" rel="preconnect"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-b92e9647318f.css" media="all" rel="stylesheet"><link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-5d486a4ede8e.css" media="all" rel="stylesheet"><link crossorigin="anonymous" data-color-theme="dark_dimmed" data-href="https://github.githubassets.com/

In [20]:
len(url)

44

In [21]:
read = []
i = 0
for readme in repos:
    url = repos[i]
    response = requests.get(url, headers={"Authorization": f"token {github_token}", "User-Agent": github_username})
    soup = BeautifulSoup(response.text, 'html.parser')
    read.append([element.text for element in soup.find_all('readme-toc')])
    i += 1

In [22]:
len(read)

107

In [23]:
read

[["\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nfreeCodeCamp.org's open-source codebase and curriculum\nTable of Contents\nCertifications\n1. Responsive Web Design Certification\n2. JavaScript Algorithms and Data Structures Certification\n3. Front End Libraries Certification\n4. Data Visualization Certification\n5. Relational Database\n6. Back End Development and APIs Certification\n7. Quality Assurance Certification\n8. Scientific Computing with Python Certification\n9. Data Analysis with Python Certification\n10. Information Security Certification\n11. Machine Learning with Python Certification\n12. College Algebra with Python\nLegacy Full Stack Development Certification\nLegacy Certifications\nFree professional certifications\n(New) Foundational C# with Microsoft\nThe Learning Platform\nJoin the community here.\nReporting Bugs and Issues\nReporting Security Issues and Responsible Disclosure\nRead our security policy and follow these steps to report a vulnerability.\nContributing\nPlease follow t

In [24]:
i = 0
for readme in repo:
    [element.text for element in soup.find_all('readme-toc')]

In [39]:
titles = []
pattern = r'[^/]+$'
for repo in repos:
    match = re.search(pattern, repo)
    last_part = match.group()
    titles.append(last_part)
titles[105] = titles[105][-22:]
titles[106] = titles[106][-10:]
titles

['freeCodeCamp',
 'free-programming-books',
 'awesome',
 'coding-interview-university',
 '996.ICU',
 'public-apis',
 'developer-roadmap',
 'system-design-primer',
 'build-your-own-x',
 'react',
 'vue',
 'core',
 'awesome-python',
 'tensorflow',
 'javascript-algorithms',
 'You-Dont-Know-JS',
 'Python',
 'CS-Notes',
 'bootstrap',
 'ohmyzsh',
 'linux',
 'flutter',
 'awesome-selfhosted',
 'gitignore',
 'vscode',
 'AutoGPT',
 'computer-science',
 'Python-100-Days',
 'the-art-of-command-line',
 'javascript',
 'JavaGuide',
 'youtube-dl',
 'fucking-algorithm',
 'project-based-learning',
 '30-seconds-of-code',
 'go',
 'transformers',
 'next.js',
 'react-native',
 'awesome-go',
 'the-book-of-secret-knowledge',
 'electron',
 'stable-diffusion-webui',
 'd3',
 'free-programming-books-zh_CN',
 'kubernetes',
 'axios',
 'create-react-app',
 'node',
 'PowerToys',
 'tech-interview-handbook',
 'three.js',
 'TypeScript',
 'nodebestpractices',
 'scrcpy',
 'deno',
 'terminal',
 'awesome-chatgpt-prompts',
 '

In [40]:
flat_list = [item for sublist in read for item in sublist]
# Convert each item to a string
flat_list_of_strings = [str(item) for item in flat_list]

In [41]:
new_dict = dict(zip(titles, flat_list_of_strings))

In [42]:
df = pd.DataFrame(list(new_dict.items()), columns=['title', 'readme'])

In [43]:
df.title = df.title.str.lower()
df.readme = df.readme.str.lower()

In [44]:
df.head()

,title,readme
0,freecodecamp,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nfreecodecamp.o...
1,free-programming-books,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nlist of free l...
2,awesome,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\none task\ncont...
3,coding-interview-university,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ncoding intervi...
4,996.icu,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n996.icu\nrelat...


In [7]:
df

,title,readme
0,freecodecamp,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nfreecodecamp.o...
1,free-programming-books,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nlist of free l...
2,awesome,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\none task\ncont...
3,coding-interview-university,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ncoding intervi...
4,996.icu,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n996.icu\nrelat...
...,...,...
96,tauri,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\njson server \n...
97,hugo,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n👇 practice dai...
98,iptv,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsystem design ...
99,json-server,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ntable of conte...


## Prep

In [11]:
df = acquire()

In [12]:
df

,title,readme
0,freecodecamp,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nfreecodecamp.o...
1,free-programming-books,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nlist of free l...
2,awesome,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\none task\ncont...
3,coding-interview-university,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ncoding intervi...
4,996.icu,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n996.icu\nrelat...
...,...,...
96,tauri,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\njson server \n...
97,hugo,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n👇 practice dai...
98,iptv,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsystem design ...
99,json-server,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ntable of conte...


In [13]:
i = 0
for read in df.readme:
    df.readme[i] = df.readme[i][16:]
    i += 1

In [14]:
i = 0
for read in df.readme:
    if "\n" in df.readme[i]:
        df.readme[i] = df.readme[i].replace('\n', ' ')
    i += 1

In [15]:
df.head()

,title,readme
0,freecodecamp,freecodecamp.org's open-source codebase and cu...
1,free-programming-books,list of free learning resources in many langua...
2,awesome,one task contents platforms programming langua...
3,coding-interview-university,coding interview university what is it? table ...
4,996.icu,996.icu related press coverage: what is 996? w...


def basic_clean(input_string):
    # Convert to lowercase
    cleaned_string = input_string.lower()
    # Normalize unicode characters
    cleaned_string = unicodedata.normalize('NFKD', cleaned_string).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # Replace anything that is not a letter, number, whitespace or a single quote
    cleaned_string = ''.join([char for char in cleaned_string if char.isalnum() or char.isspace() or char == "'"])
    return cleaned_string

i = 0
for element in df.readme:
    df.readme[i] = basic_clean(element)
    i += 1

df.head()

def tokenize(string_cleaning):
    '''
    tokenize will take in a string and return the string tokenized
    '''
    tokenizer = nltk.tokenize.ToktokTokenizer()
    return tokenizer.tokenize(string_cleaning, return_str=True)

tok_clean = [tokenize(el) for el in df.readme]
tok_clean

def remove_stopwords(string_cleaning, extra_words='', exclude_words=''):
    '''
    remove_stopwords will take in a string, any extra words to be removed, and and words to keep, and return the string with stop words removed
    '''
    stopword_list = stopwords.words('english')
    stopword_list = stopword_list.remove(exclude_words)
    stopword_list = [stopword_list] + [extra_words]
    words = string_cleaning.split()
    filtered_words = [w for w in words if w not in stopword_list]
    string_without_stopwords = ' '.join(filtered_words)
    return string_without_stopwords

df.title

In [16]:
ADDITIONAL_STOPWORDS = ['python', 'java', 'r'] + [ti for ti in df.title]
def clean(text):
    'A simple function to cleanup text data'
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    text = (unicodedata.normalize('NFKD', text)
             .encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [17]:
df['lemmatized'] = [clean(readme) for readme in df.readme]

In [18]:
df.head()

,title,readme,lemmatized
0,freecodecamp,freecodecamp.org's open-source codebase and cu...,"[freecodecamporgs, opensource, codebase, curri..."
1,free-programming-books,list of free learning resources in many langua...,"[list, free, learning, resource, many, languag..."
2,awesome,one task contents platforms programming langua...,"[one, task, content, platform, programming, la..."
3,coding-interview-university,coding interview university what is it? table ...,"[coding, interview, university, table, content..."
4,996.icu,996.icu related press coverage: what is 996? w...,"[996icu, related, press, coverage, 996, voice,..."


In [3]:
df = acquire()
df.head()

,title,readme
0,freecodecamp,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nfreecodecamp.o...
1,free-programming-books,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nlist of free l...
2,awesome,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\none task\ncont...
3,coding-interview-university,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ncoding intervi...
4,996.icu,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n996.icu\nrelat...


In [5]:
df = prep(df)
df.head()

NameError: name 'df' is not defined

In [135]:
df

,title,readme,lemmatized
0,freecodecamp,freecodecamporg's opensource codebase and curr...,"[freecodecamporgs, opensource, codebase, curri..."
1,free-programming-books,list of free learning resources in many langua...,"[list, free, learning, resource, many, languag..."
2,awesome,one task contents platforms programming langua...,"[one, task, content, platform, programming, la..."
3,coding-interview-university,coding interview university what is it table o...,"[coding, interview, university, table, content..."
4,996.icu,996icu related press coverage what is 996 what...,"[996icu, related, press, coverage, 996, voice,..."
...,...,...,...
96,tauri,json server gold sponsors silver sponsors s...,"[json, server, gold, sponsor, silver, sponsor,..."
97,hugo,practice daily on icodethiscom app ideas co...,"[practice, daily, icodethiscom, app, idea, col..."
98,iptv,system design 101 table of contents communicat...,"[system, design, 101, table, content, communic..."
99,json-server,table of contents what is openpilot running on...,"[table, content, openpilot, running, dedicated..."


In [91]:
df

,title,readme,lemmatized
0,freecodecamp,freecodecamporg's opensource codebase and curr...,"[freecodecamporgs, opensource, codebase, curri..."
1,free-programming-books,list of free learning resources in many langua...,"[list, free, learning, resource, many, languag..."
2,awesome,one task contents platforms programming langua...,"[one, task, content, platform, programming, la..."
3,coding-interview-university,coding interview university what is it table o...,"[coding, interview, university, table, content..."
4,996.icu,996icu related press coverage what is 996 what...,"[996icu, related, press, coverage, 996, voice,..."
...,...,...,...
96,tauri,json server gold sponsors silver sponsors s...,"[json, server, gold, sponsor, silver, sponsor,..."
97,hugo,practice daily on icodethiscom app ideas co...,"[practice, daily, icodethiscom, app, idea, col..."
98,iptv,system design 101 table of contents communicat...,"[system, design, 101, table, content, communic..."
99,json-server,table of contents what is openpilot running on...,"[table, content, openpilot, running, dedicated..."


In [ ]:
pd.Series([clean(readme) for readme in df.readme][1]).value_counts()

list           8
language       8
free           8
programming    8
portuguese     7
              ..
remember       1
abide          1
adapted        1
also           1
cc             1
Name: count, Length: 209, dtype: int64

In [ ]:
stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS

In [ ]:
[ti for ti in df.title]

['freecodecamp',
 'free-programming-books',
 'awesome',
 'coding-interview-university',
 '996.icu',
 'public-apis',
 'developer-roadmap',
 'system-design-primer',
 'build-your-own-x',
 'react',
 'vue',
 'core',
 'awesome-python',
 'tensorflow',
 'javascript-algorithms',
 'you-dont-know-js',
 'python',
 'cs-notes',
 'bootstrap',
 'ohmyzsh',
 'linux',
 'flutter',
 'awesome-selfhosted',
 'gitignore',
 'vscode',
 'autogpt',
 'computer-science',
 'python-100-days',
 'the-art-of-command-line',
 'javascript',
 'javaguide',
 'youtube-dl',
 'fucking-algorithm',
 'project-based-learning',
 '30-seconds-of-code',
 'go',
 'transformers',
 'next.js',
 'react-native',
 'awesome-go',
 'the-book-of-secret-knowledge',
 'electron',
 'stable-diffusion-webui',
 'd3',
 'free-programming-books-zh_cn',
 'kubernetes',
 'axios',
 'create-react-app',
 'node',
 'powertoys',
 'tech-interview-handbook',
 'three.js',
 'typescript',
 'nodebestpractices',
 'scrcpy',
 'deno',
 'terminal',
 'awesome-chatgpt-prompts',
 '

In [36]:
df = wrangle()

In [44]:
df[50:75]

,title,readme,lemmatized
50,tech-interview-handbook,ee.js javascript 3d library usage cloning this...,"[eejs, 3d, library, usage, cloning, repository..."
51,three.js,escript installing contribute documentation ro...,"[escript, installing, contribute, documentatio..."
52,typescript,node.js best practices 🎊 2023 edition is here!...,"[nodejs, best, practice, 2023, edition, welcom..."
53,nodebestpractices,scrcpy (v2.1.1) prerequisites get the app user...,"[v211, prerequisite, get, app, user, documenta..."
54,scrcpy,o features install getting started additional ...,"[feature, install, getting, started, additiona..."
55,deno,"welcome to the windows terminal, console and c...","[welcome, window, console, commandline, repo, ..."
56,terminal,🧠 awesome chatgpt prompts sponsors want to wri...,"[chatgpt, prompt, sponsor, want, write, effect..."
57,awesome-chatgpt-prompts,angular - the modern web developer's platform ...,"[modern, web, developer, platform, documentati..."
58,angular,mui core documentation material ui joy ui base...,"[mui, documentation, material, ui, joy, ui, ba..."
59,material-ui,ant design ✨ features 🖥 environment support 📦 ...,"[ant, design, feature, environment, support, i..."


In [48]:
df = wrangle()
df.head()

,title,readme,lemmatized
0,freecodecamp,freecodecamp.org's open-source codebase and cu...,freecodecamporgs opensource codebase curriculu...
1,free-programming-books,list of free learning resources in many langua...,freecodecamporgs opensource codebase curriculu...
2,awesome,one task contents platforms programming langua...,freecodecamporgs opensource codebase curriculu...
3,coding-interview-university,coding interview university what is it? table ...,freecodecamporgs opensource codebase curriculu...
4,996.icu,996.icu related press coverage: what is 996? w...,freecodecamporgs opensource codebase curriculu...


## Exploration

In [492]:
df = wrangle()

In [493]:
df

,title,readme,lemmatized
0,freecodecamp,freecodecamp.org's open-source codebase and cu...,freecodecamporgs opensource codebase curriculu...
1,free-programming-books,list of free learning resources in many langua...,list free learning resource many language intr...
2,awesome,one task contents platforms programming langua...,one task content platform programming language...
3,coding-interview-university,coding interview university what is it? table ...,coding interview university table content stud...
4,996.icu,996.icu related press coverage: what is 996? w...,996icu related press coverage 996 voice state ...
...,...,...,...
96,tauri,json server gold sponsors 🥇 silver sponsors 🥈...,json server gold sponsor silver sponsor sponso...
97,hugo,👇 practice daily on icodethis.com 👇 📒 app idea...,practice daily icodethiscom app idea collectio...
98,iptv,system design 101 table of contents communicat...,system design 101 table content communication ...
99,json-server,table of contents what is openpilot? running o...,table content openpilot running dedicated devi...


In [494]:
df['lemmatized']

0      freecodecamporgs opensource codebase curriculu...
1      list free learning resource many language intr...
2      one task content platform programming language...
3      coding interview university table content stud...
4      996icu related press coverage 996 voice state ...
                             ...                        
96     json server gold sponsor silver sponsor sponso...
97     practice daily icodethiscom app idea collectio...
98     system design 101 table content communication ...
99     table content openpilot running dedicated devi...
100    localsend screenshots download work getting st...
Name: lemmatized, Length: 101, dtype: object

def tokenize(string):
    tokenizer = nltk.tokenize.ToktokTokenizer()
    string = tokenizer.tokenize(string, return_str=True)
    return string

df['lemmatized'] = df['lemmatized'].apply(tokenize)

df['lemmatized'].str.split()

In [495]:
language = ['TypeScript', 'None', 'None', 'None', 'None', 'Python', 'TypeScript', 'Python', 'None', 'JavaScript', 'TypeScript', 'Python', 'C++', 'JavaScript', 'None', 'Python', 'None', 'JavaScript', 'Shell', 
           'C', 'Dart', 'None', 'None', 'TypeScript', 'JavaScript', 'None', 'Python', 'None', 'JavaScript', 'Java', 'Python', 'Markdown', 'None', 'JavaScript', 'JavaScript', 'Go', 'Python', 'JavaScript', 'Java', 
            'Go', 'None', 'C++', 'Python', 'Shell', 'None', 'Go', 'JavaScript', 'JavaScript', 'JavaScript', 'C#', 'TypeScript', 'JavaScript', 'TypeScript', 'Dockerfile', 'C', 'Rust', 'C++', 'HTML', 'TypeScript', 
            'TypeScript', 'TypeScript', 'Rust', 'JavaScript', 'TypeScript', 'Rust', 'JavaScript', 'TypeScript', 'Vue', 'Java', 'TypeScript', 'Python', 'CSS', 'JavaScript', 'Shell', 'Python', 'TypeScript', 'C++', 
            'Jupyter Notebook', 'html', 'php', 'java', 'python', 'none', 'javascript', 'html', 'go', 'clojure', 'javascript', 'go', 'java', 'c++', 'python', 'c++', 'shell', 'java', 'vim script', 'none', 'java', 
            'none', 'go', 'rust']

In [496]:
df['language'] = language

In [497]:
df['language'] = df.language.str.lower()
df

,title,readme,lemmatized,language
0,freecodecamp,freecodecamp.org's open-source codebase and cu...,freecodecamporgs opensource codebase curriculu...,typescript
1,free-programming-books,list of free learning resources in many langua...,list free learning resource many language intr...,none
2,awesome,one task contents platforms programming langua...,one task content platform programming language...,none
3,coding-interview-university,coding interview university what is it? table ...,coding interview university table content stud...,none
4,996.icu,996.icu related press coverage: what is 996? w...,996icu related press coverage 996 voice state ...,none
...,...,...,...,...
96,tauri,json server gold sponsors 🥇 silver sponsors 🥈...,json server gold sponsor silver sponsor sponso...,none
97,hugo,👇 practice daily on icodethis.com 👇 📒 app idea...,practice daily icodethiscom app idea collectio...,java
98,iptv,system design 101 table of contents communicat...,system design 101 table content communication ...,none
99,json-server,table of contents what is openpilot? running o...,table content openpilot running dedicated devi...,go


In [498]:
df['lemmatized'] = df['lemmatized'].str.split()

def tokenize(string):
    tokenizer = nltk.tokenize.ToktokTokenizer()
    string = tokenizer.tokenize(string, return_str=True)
    return string
tok_clean = ' '.join([tokenize(clean) for clean in df.lemmatized]).split()

tok_clean

In [499]:
df

,title,readme,lemmatized,language
0,freecodecamp,freecodecamp.org's open-source codebase and cu...,"[freecodecamporgs, opensource, codebase, curri...",typescript
1,free-programming-books,list of free learning resources in many langua...,"[list, free, learning, resource, many, languag...",none
2,awesome,one task contents platforms programming langua...,"[one, task, content, platform, programming, la...",none
3,coding-interview-university,coding interview university what is it? table ...,"[coding, interview, university, table, content...",none
4,996.icu,996.icu related press coverage: what is 996? w...,"[996icu, related, press, coverage, 996, voice,...",none
...,...,...,...,...
96,tauri,json server gold sponsors 🥇 silver sponsors 🥈...,"[json, server, gold, sponsor, silver, sponsor,...",none
97,hugo,👇 practice daily on icodethis.com 👇 📒 app idea...,"[practice, daily, icodethiscom, app, idea, col...",java
98,iptv,system design 101 table of contents communicat...,"[system, design, 101, table, content, communic...",none
99,json-server,table of contents what is openpilot? running o...,"[table, content, openpilot, running, dedicated...",go


In [500]:
df.language.value_counts()

language
javascript          17
none                17
typescript          13
python              12
java                 7
go                   6
c++                  6
shell                4
rust                 4
html                 3
c                    2
dart                 1
markdown             1
c#                   1
dockerfile           1
vue                  1
css                  1
jupyter notebook     1
php                  1
clojure              1
vim script           1
Name: count, dtype: int64

In [501]:
df.language = df.language.map({'javascript':'java', 'java':'java', 'c':'c', 'shell':'other', 'none':'other', 'c++':'c', 'typescript':'typescript',
                               'c#':'c', 'go':'other', 'rust':'other', 'html':'other', 'dart':'other', 'python':'python',
                               'markdown':'other', 'dockerfile':'other', 'vue':'other', 'css':'c', 
                               'jupyter notebook':'other', 'php':'other', 'clojure':'other', 'vim script':'other'})

In [502]:
df.language.value_counts(normalize=True)

language
other         0.415842
java          0.237624
typescript    0.128713
python        0.118812
c             0.099010
Name: proportion, dtype: float64

In [503]:
java_words = df[df['language'] == 'java']['lemmatized']
split_java_words = ' '.join(java_words.apply(lambda x: ' '.join(x))).split()
python_words = df[df['language'] == 'python']['lemmatized']
split_python_words = ' '.join(python_words.apply(lambda x: ' '.join(x))).split()
typescript_words = df[df['language'] == 'typescript']['lemmatized']
split_typescript_words = ' '.join(typescript_words.apply(lambda x: ' '.join(x))).split()
c_words = df[df['language'] == 'c']['lemmatized']
split_c_words = ' '.join(c_words.apply(lambda x: ' '.join(x))).split()
other_words = df[df['language'] == 'other']['lemmatized']
split_other_words = ' '.join(other_words.apply(lambda x: ' '.join(x))).split()
all_words = df['lemmatized']
split_all_words = ' '.join(all_words.apply(lambda x: ' '.join(x))).split()

Java

In [504]:
java_bigrams = list(nltk.ngrams(split_java_words,2))
top_20_java_bigrams = (pd.Series(nltk.ngrams(split_java_words, 2)).value_counts().head(20))

java_trigrams = list(nltk.ngrams(split_java_words,3))
top_20_java_trigrams = (pd.Series(nltk.ngrams(split_java_words, 3)).value_counts().head(20))

Bigrams

In [505]:
java_bigrams[:5]

[('installation', 'documentation'),
 ('documentation', 'example'),
 ('example', 'contributing'),
 ('contributing', 'conduct'),
 ('conduct', 'contributing')]

In [506]:
top_20_java_bigrams[:5]

(back, top)          124
(bad, const)          71
(good, const)         64
(command, line)       51
(client, library)     47
Name: count, dtype: int64

Trigrams

In [507]:
java_trigrams[:5]

[('installation', 'documentation', 'example'),
 ('documentation', 'example', 'contributing'),
 ('example', 'contributing', 'conduct'),
 ('contributing', 'conduct', 'contributing'),
 ('conduct', 'contributing', 'guide')]

In [508]:
top_20_java_trigrams[:5]

(command, line, tool)    14
(1, 2, 3)                12
(const, x, 1)            12
(good, const, foo)       11
(bad, const, foo)        11
Name: count, dtype: int64

In [509]:
java_data_2 = {k[0] + ' ' + k[1]: v for k, v in top_20_java_bigrams.to_dict().items()}
java_img_2 = WordCloud(background_color='white', width=750, height=500).generate_from_frequencies(java_data_2)

java_data_3 = {k[0] + ' ' + k[1] + ' ' + k[2]: v for k, v in top_20_java_trigrams.to_dict().items()}
java_img_3 = WordCloud(background_color='white', width=750, height=500).generate_from_frequencies(java_data_3)

Python

In [510]:
python_bigrams = list(nltk.ngrams(split_python_words,2))
top_20_python_bigrams = (pd.Series(nltk.ngrams(split_python_words, 2)).value_counts().head(20))

python_trigrams = list(nltk.ngrams(split_python_words,3))
top_20_python_trigrams = (pd.Series(nltk.ngrams(split_python_words, 3)).value_counts().head(20))

Bigrams

In [511]:
python_bigrams[:5]

[('public', 'apis'),
 ('apis', 'index'),
 ('index', 'animal'),
 ('animal', 'anime'),
 ('anime', 'antimalware')]

Trigrams

In [512]:
python_trigrams[:5]

[('public', 'apis', 'index'),
 ('apis', 'index', 'animal'),
 ('index', 'animal', 'anime'),
 ('animal', 'anime', 'antimalware'),
 ('anime', 'antimalware', 'art')]

In [513]:
python_data_2 = {k[0] + ' ' + k[1]: v for k, v in top_20_python_bigrams.to_dict().items()}
python_img_2 = WordCloud(background_color='white', width=750, height=500).generate_from_frequencies(python_data_2)

python_data_3 = {k[0] + ' ' + k[1] + ' ' + k[2]: v for k, v in top_20_python_trigrams.to_dict().items()}
python_img_3 = WordCloud(background_color='white', width=750, height=500).generate_from_frequencies(python_data_3)

Typescript

In [514]:
typescript_bigrams = list(nltk.ngrams(split_typescript_words,2))
top_20_typescript_bigrams = (pd.Series(nltk.ngrams(split_typescript_words, 2)).value_counts().head(20))

typescript_trigrams = list(nltk.ngrams(split_typescript_words,3))
top_20_typescript_trigrams = (pd.Series(nltk.ngrams(split_typescript_words, 3)).value_counts().head(20))

Bigrams

In [515]:
typescript_bigrams[:5]

[('freecodecamporgs', 'opensource'),
 ('opensource', 'codebase'),
 ('codebase', 'curriculum'),
 ('curriculum', 'table'),
 ('table', 'content')]

Trigrams

In [516]:
typescript_trigrams[:5]

[('freecodecamporgs', 'opensource', 'codebase'),
 ('opensource', 'codebase', 'curriculum'),
 ('codebase', 'curriculum', 'table'),
 ('curriculum', 'table', 'content'),
 ('table', 'content', 'certification')]

In [517]:
typescript_data_2 = {k[0] + ' ' + k[1]: v for k, v in top_20_typescript_bigrams.to_dict().items()}
typescript_img_2 = WordCloud(background_color='white', width=750, height=500).generate_from_frequencies(typescript_data_2)

typescript_data_3 = {k[0] + ' ' + k[1] + ' ' + k[2]: v for k, v in top_20_typescript_trigrams.to_dict().items()}
typescript_img_3 = WordCloud(background_color='white', width=750, height=500).generate_from_frequencies(typescript_data_3)

C

In [518]:
c_bigrams = list(nltk.ngrams(split_c_words,2))
top_20_c_bigrams = (pd.Series(nltk.ngrams(split_c_words, 2)).value_counts().head(20))

c_trigrams = list(nltk.ngrams(split_c_words,3))
top_20_c_trigrams = (pd.Series(nltk.ngrams(split_c_words, 3)).value_counts().head(20))

Bigrams

In [519]:
top_20_c_bigrams[:5]

(want, act)                  146
(vuejs, 2)                    84
(component, vuejs)            77
(contributed, devisasari)     73
(devisasari, want)            69
Name: count, dtype: int64

Trigrams

In [520]:
top_20_c_trigrams[:5]

(contributed, devisasari, want)    69
(devisasari, want, act)            67
(request, need, help)              42
(first, suggestion, request)       31
(first, request, need)             28
Name: count, dtype: int64

In [521]:
c_data_2 = {k[0] + ' ' + k[1]: v for k, v in top_20_c_bigrams.to_dict().items()}
c_img_2 = WordCloud(background_color='white', width=750, height=500).generate_from_frequencies(c_data_2)

c_data_3 = {k[0] + ' ' + k[1] + ' ' + k[2]: v for k, v in top_20_c_trigrams.to_dict().items()}
c_img_3 = WordCloud(background_color='white', width=750, height=500).generate_from_frequencies(c_data_3)

Other

In [522]:
other_bigrams = list(nltk.ngrams(split_other_words,2))
top_20_other_bigrams = (pd.Series(nltk.ngrams(split_other_words, 2)).value_counts().head(20))

other_trigrams = list(nltk.ngrams(split_other_words,3))
top_20_other_trigrams = (pd.Series(nltk.ngrams(split_other_words, 3)).value_counts().head(20))

In [523]:
other_data_2 = {k[0] + ' ' + k[1]: v for k, v in top_20_other_bigrams.to_dict().items()}
other_img_2 = WordCloud(background_color='white', width=750, height=500).generate_from_frequencies(other_data_2)

other_data_3 = {k[0] + ' ' + k[1] + ' ' + k[2]: v for k, v in top_20_other_trigrams.to_dict().items()}
other_img_3 = WordCloud(background_color='white', width=750, height=500).generate_from_frequencies(other_data_3)

All

In [524]:
all_bigrams = list(nltk.ngrams(split_all_words,2))
top_20_all_bigrams = (pd.Series(nltk.ngrams(split_all_words, 2)).value_counts().head(20))

all_trigrams = list(nltk.ngrams(split_all_words,3))
top_20_all_trigrams = (pd.Series(nltk.ngrams(split_all_words, 3)).value_counts().head(20))

all_data_2 = {k[0] + ' ' + k[1]: v for k, v in top_20_all_bigrams.to_dict().items()}
all_img_2 = WordCloud(background_color='white', width=750, height=500).generate_from_frequencies(all_data_2)

all_data_3 = {k[0] + ' ' + k[1] + ' ' + k[2]: v for k, v in top_20_all_trigrams.to_dict().items()}
all_img_3 = WordCloud(background_color='white', width=750, height=500).generate_from_frequencies(all_data_3)

plt.figure(figsize=(12, 8))
axs = [plt.axes([0, .5, .25, .5]), plt.axes([0, 0, .25, .85]), plt.axes([.25, .5, .25, .5]), plt.axes([.25, 0, .25, .85]),
      plt.axes([.5, .5, .25, .5]), plt.axes([.5, 0, .25, .85]), plt.axes([.75, .5, .25, .5]), plt.axes([.75, 0, .25, .85])]

axs[0].imshow(java_img_2)
axs[1].imshow(java_img_3)
axs[2].imshow(python_img_2)
axs[3].imshow(python_img_3)
axs[4].imshow(typescript_img_2)
axs[5].imshow(typescript_img_3)
axs[6].imshow(c_img_2)
axs[7].imshow(c_img_3)
axs[8].imshow(other_img_2)
axs[9].imshow(other_img_3)
axs[10].imshow(all_img_2)
axs[11].imshow(all_img_3)

axs[0].set_title('Java Bigram Word Cloud')
axs[1].set_title('Java Trigram Word Cloud')
axs[2].set_title('Python Bigram Word Cloud')
axs[3].set_title('Python Trigram Word Cloud')
axs[4].set_title('Typescript Bigram Word Cloud')
axs[5].set_title('Typescript Trigram Word Cloud')
axs[6].set_title('C Bigram Word Cloud')
axs[7].set_title('C Trigram Word Cloud')
axs[8].set_title('Other Bigram Word Cloud')
axs[9].set_title('Other Trigram Word Cloud')
axs[10].set_title('All Bigram Word Cloud')
axs[11].set_title('All Trigram Word Cloud')

for ax in axs: ax.axis('off')
plt.show()

## Splitting

In [525]:
train['baseline_prediction'] = 'other'
baseline_accuracy = (train.baseline_prediction == train.language).mean()
print(f'baseline accuracy: {baseline_accuracy:.2%}')

baseline accuracy: 50.00%


In [526]:
X = df.lemmatized
y = df.language

Stats Testing

In [527]:
java_words = df[df['language'] == 'java']['lemmatized']
split_java_words = ' '.join(java_words.apply (lambda x: ' '.join(x))).split()
python_words = df[df['language'] == 'python']['lemmatized']
split_python_words = ' '.join(python_words.apply (lambda x: ' '.join(x))).split()
typescript_words = df[df['language'] == 'typescript']['lemmatized']
split_typescript_words = ' '.join(typescript_words.apply (lambda x: ' '.join(x))).split()
c_words = df[df['language'] == 'c']['lemmatized']
split_c_words = ' '.join(c_words.apply (lambda x: ' '.join(x))).split()
other_words = df[df['language'] == 'other']['lemmatized']
split_other_words = ' '.join(other_words.apply (lambda x: ' '.join(x))).split()
all_words = df['lemmatized']
split_all_words = ' '.join(all_words.apply (lambda x: ' '.join(x))).split()

In [528]:
python_bigrams = list(nltk.ngrams(split_python_words,2))
top_20_python_bigrams = (pd.Series(nltk.ngrams(split_python_words, 2)).value_counts().head(20))

java_bigrams = list(nltk.ngrams(split_java_words,2))
top_20_java_bigrams = (pd.Series(nltk.ngrams(split_java_words, 2)).value_counts().head(20))

typescript_bigrams = list(nltk.ngrams(split_typescript_words,2))
top_20_typescript_bigrams = (pd.Series(nltk.ngrams(split_typescript_words, 2)).value_counts().head(20))

c_bigrams = list(nltk.ngrams(split_c_words,2))
top_20_c_bigrams = (pd.Series(nltk.ngrams(split_c_words, 2)).value_counts().head(20))

other_bigrams = list(nltk.ngrams(split_other_words,2))
top_20_other_bigrams = (pd.Series(nltk.ngrams(split_other_words, 2)).value_counts().head(20))

all_bigrams = list(nltk.ngrams(split_all_words,2))
top_20_all_bigrams = (pd.Series(nltk.ngrams(split_all_words, 2)).value_counts().head(20))

In [529]:
python_bigrams

[('public', 'apis'),
 ('apis', 'index'),
 ('index', 'animal'),
 ('animal', 'anime'),
 ('anime', 'antimalware'),
 ('antimalware', 'art'),
 ('art', 'design'),
 ('design', 'authentication'),
 ('authentication', 'authorization'),
 ('authorization', 'blockchain'),
 ('blockchain', 'book'),
 ('book', 'business'),
 ('business', 'calendar'),
 ('calendar', 'cloud'),
 ('cloud', 'storage'),
 ('storage', 'file'),
 ('file', 'sharing'),
 ('sharing', 'continuous'),
 ('continuous', 'integration'),
 ('integration', 'cryptocurrency'),
 ('cryptocurrency', 'currency'),
 ('currency', 'exchange'),
 ('exchange', 'data'),
 ('data', 'validation'),
 ('validation', 'development'),
 ('development', 'dictionary'),
 ('dictionary', 'document'),
 ('document', 'productivity'),
 ('productivity', 'email'),
 ('email', 'entertainment'),
 ('entertainment', 'environment'),
 ('environment', 'event'),
 ('event', 'finance'),
 ('finance', 'food'),
 ('food', 'drink'),
 ('drink', 'game'),
 ('game', 'comic'),
 ('comic', 'geocoding'

In [530]:
df.head()

,title,readme,lemmatized,language
0,freecodecamp,freecodecamp.org's open-source codebase and cu...,"[freecodecamporgs, opensource, codebase, curri...",typescript
1,free-programming-books,list of free learning resources in many langua...,"[list, free, learning, resource, many, languag...",other
2,awesome,one task contents platforms programming langua...,"[one, task, content, platform, programming, la...",other
3,coding-interview-university,coding interview university what is it? table ...,"[coding, interview, university, table, content...",other
4,996.icu,996.icu related press coverage: what is 996? w...,"[996icu, related, press, coverage, 996, voice,...",other


Need to encode top 5 or so words/bigrams/trigrams and compare that to all words for statistical significance.

In [531]:
df

,title,readme,lemmatized,language
0,freecodecamp,freecodecamp.org's open-source codebase and cu...,"[freecodecamporgs, opensource, codebase, curri...",typescript
1,free-programming-books,list of free learning resources in many langua...,"[list, free, learning, resource, many, languag...",other
2,awesome,one task contents platforms programming langua...,"[one, task, content, platform, programming, la...",other
3,coding-interview-university,coding interview university what is it? table ...,"[coding, interview, university, table, content...",other
4,996.icu,996.icu related press coverage: what is 996? w...,"[996icu, related, press, coverage, 996, voice,...",other
...,...,...,...,...
96,tauri,json server gold sponsors 🥇 silver sponsors 🥈...,"[json, server, gold, sponsor, silver, sponsor,...",other
97,hugo,👇 practice daily on icodethis.com 👇 📒 app idea...,"[practice, daily, icodethiscom, app, idea, col...",java
98,iptv,system design 101 table of contents communicat...,"[system, design, 101, table, content, communic...",other
99,json-server,table of contents what is openpilot? running o...,"[table, content, openpilot, running, dedicated...",other


train['free'] = 'False'
train['free'] = train['lemmatized'].str.contains('free', case=False)

In [532]:
df.head()

,title,readme,lemmatized,language
0,freecodecamp,freecodecamp.org's open-source codebase and cu...,"[freecodecamporgs, opensource, codebase, curri...",typescript
1,free-programming-books,list of free learning resources in many langua...,"[list, free, learning, resource, many, languag...",other
2,awesome,one task contents platforms programming langua...,"[one, task, content, platform, programming, la...",other
3,coding-interview-university,coding interview university what is it? table ...,"[coding, interview, university, table, content...",other
4,996.icu,996.icu related press coverage: what is 996? w...,"[996icu, related, press, coverage, 996, voice,...",other


In [533]:
five_python = pd.Series(split_python_words).value_counts().head().index
five_java = pd.Series(split_java_words).value_counts().head().index
five_typescript = pd.Series(split_typescript_words).value_counts().head().index
five_c = pd.Series(split_c_words).value_counts().head().index
five_other = pd.Series(split_other_words).value_counts().head().index

In [534]:
twenty_five = five_python.append(five_java)
twenty_five = twenty_five.append(five_typescript)
twenty_five = twenty_five.append(five_c)
twenty_five = twenty_five.append(five_other)

In [535]:
twenty_five

Index(['unknown', 'apikey', 'data', 'api', 'video', 'library', 'const',
       'function', 'golang', 'use', 'using', 'error', 'read', 'use', 'tldr',
       'vuejs', 'component', 'act', 'library', 'simple', 'free', 'source',
       'data', 'tool', 'web'],
      dtype='object')

In [536]:
for i in range(len(twenty_five)):
    df[f'{twenty_five[i]}'] = df['lemmatized'].apply(lambda x: 1 if twenty_five[i] in x else 0)

In [537]:
df.head()

,title,readme,lemmatized,language,unknown,apikey,data,api,video,library,...,read,tldr,vuejs,component,act,simple,free,source,tool,web
0,freecodecamp,freecodecamp.org's open-source codebase and cu...,"[freecodecamporgs, opensource, codebase, curri...",typescript,0,0,1,0,0,1,...,1,0,0,0,0,1,1,0,1,1
1,free-programming-books,list of free learning resources in many langua...,"[list, free, learning, resource, many, languag...",other,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,awesome,one task contents platforms programming langua...,"[one, task, content, platform, programming, la...",other,0,0,1,1,1,1,...,1,0,1,1,0,1,1,1,1,1
3,coding-interview-university,coding interview university what is it? table ...,"[coding, interview, university, table, content...",other,0,0,1,0,1,0,...,1,0,0,1,0,1,1,1,1,1
4,996.icu,996.icu related press coverage: what is 996? w...,"[996icu, related, press, coverage, 996, voice,...",other,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [538]:
df['is_java'] = df['language'] == 'java'
df['is_java'] = df['is_java'].map({False:0, True:1})
df['is_python'] = df['language'] == 'python'
df['is_python'] = df['is_python'].map({False:0, True:1})
df['is_typescript'] = df['language'] == 'typescript'
df['is_typescript'] = df['is_typescript'].map({False:0, True:1})
df['is_c'] = df['language'] == 'c'
df['is_c'] = df['is_c'].map({False:0, True:1})
df['is_other'] = df['language'] == 'other'
df['is_other'] = df['is_other'].map({False:0, True:1})

In [539]:
def train_val_test(df):
    train_val, test = train_test_split(df,
                                  random_state=1349,
                                  train_size=0.8)
    train, validate = train_test_split(train_val,
                                  random_state=1349,
                                  train_size=0.7)
    return train, validate, test
train, validate, test = train_val_test(df)
train.head()

,title,readme,lemmatized,language,unknown,apikey,data,api,video,library,...,simple,free,source,tool,web,is_java,is_python,is_typescript,is_c,is_other
30,javaguide,installation description options network optio...,"[installation, description, option, network, o...",python,0,0,1,1,1,0,...,1,1,1,0,1,0,1,0,0,0
13,tensorflow,install try your first tensorflow program cont...,"[install, try, first, program, contribution, g...",java,0,0,0,1,0,1,...,0,0,1,1,0,1,0,0,0,0
24,vscode,🌟 autogpt: the heart of the open-source agent ...,"[heart, opensource, agent, ecosystem, current,...",java,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
99,json-server,table of contents what is openpilot? running o...,"[table, content, openpilot, running, dedicated...",other,0,0,1,0,0,1,...,0,1,1,1,0,0,0,0,0,1
50,tech-interview-handbook,ee.js javascript 3d library usage cloning this...,"[eejs, 3d, library, usage, cloning, repository...",typescript,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [540]:
pd.Series(split_other_words).value_counts().head().index

Index(['free', 'source', 'data', 'tool', 'web'], dtype='object')

In [541]:
pd.Series(split_all_words).value_counts().head().index

Index(['source', 'free', 'data', 'api', 'file'], dtype='object')

In [542]:
pd.Series(split_other_words).value_counts().head()

free      1346
source    1231
data       628
tool       607
web        598
Name: count, dtype: int64

In [543]:
from scipy import stats

In [544]:
observed = pd.crosstab(train['unknown'], train.language)
chi2, p, degf, expected = stats.chi2_contingency(observed)
alpha = 0.05
if p < alpha:
    print('Reject the null hypothesis. Meaning, There is an association between contract type and churn. They are not independent.')
else:
    print('Fail to reject the null. Meaning, There is not an association between contract type and churn. They are independent.')

Fail to reject the null. Meaning, There is not an association between contract type and churn. They are independent.


In [545]:
observed

language,c,java,other,python,typescript
unknown,,,,,
0,4,10,28,4,7
1,0,2,0,1,0


In [546]:
observed = pd.crosstab(train['free'], train['language'])
chi2, p, degf, expected = stats.chi2_contingency(observed)
alpha = 0.05
if p < alpha:
    print('Reject the null hypothesis. Meaning, There is an association between contract type and churn. They are not independent.')
else:
    print('Fail to reject the null. Meaning, There is not an association between contract type and churn. They are independent.')

Reject the null hypothesis. Meaning, There is an association between contract type and churn. They are not independent.


In [547]:
observed

language,c,java,other,python,typescript
free,,,,,
0,1,11,14,2,7
1,3,1,14,3,0


In [548]:
observed = pd.crosstab(train['source'], train['language'])
chi2, p, degf, expected = stats.chi2_contingency(observed)
alpha = 0.05
if p < alpha:
    print('Reject the null hypothesis. Meaning, There is an association between contract type and churn. They are not independent.')
else:
    print('Fail to reject the null. Meaning, There is not an association between contract type and churn. They are independent.')

Fail to reject the null. Meaning, There is not an association between contract type and churn. They are independent.


In [549]:
observed

language,c,java,other,python,typescript
source,,,,,
0,0,4,7,1,4
1,4,8,21,4,3


In [550]:
observed = pd.crosstab(train['data'], train['language'])
chi2, p, degf, expected = stats.chi2_contingency(observed)
alpha = 0.05
if p < alpha:
    print('Reject the null hypothesis. Meaning, There is an association between contract type and churn. They are not independent.')
else:
    print('Fail to reject the null. Meaning, There is not an association between contract type and churn. They are independent.')

Fail to reject the null. Meaning, There is not an association between contract type and churn. They are independent.


In [551]:
observed

language,c,java,other,python,typescript
data,,,,,
0,1,10,13,3,6
1,3,2,15,2,1


In [552]:
observed = pd.crosstab(train['api'], train['language'])
chi2, p, degf, expected = stats.chi2_contingency(observed)
alpha = 0.05
if p < alpha:
    print('Reject the null hypothesis. Meaning, There is an association between contract type and churn. They are not independent.')
else:
    print('Fail to reject the null. Meaning, There is not an association between contract type and churn. They are independent.')

Fail to reject the null. Meaning, There is not an association between contract type and churn. They are independent.


In [553]:
observed

language,c,java,other,python,typescript
api,,,,,
0,3,6,12,3,5
1,1,6,16,2,2


In [554]:
observed = pd.crosstab(train['vuejs'], train['language'])
chi2, p, degf, expected = stats.chi2_contingency(observed)
alpha = 0.05
if p < alpha:
    print('Reject the null hypothesis. Meaning, There is an association between contract type and churn. They are not independent.')
else:
    print('Fail to reject the null. Meaning, There is not an association between contract type and churn. They are independent.')

Fail to reject the null. Meaning, There is not an association between contract type and churn. They are independent.


In [555]:
observed

language,c,java,other,python,typescript
vuejs,,,,,
0,3,11,23,4,6
1,1,1,5,1,1


In [556]:
for word in twenty_five:
    observed = pd.crosstab(train[word], train.language)
    chi2, p, degf, expected = stats.chi2_contingency(observed)
    alpha = 0.05
    if p < alpha:
        print(f'Reject the null hypothesis. Meaning, There is an association between "{word}" and language. They are not independent.')
    else:
        print(f'Fail to reject null')
    print()

Fail to reject null

Reject the null hypothesis. Meaning, There is an association between "apikey" and language. They are not independent.

Fail to reject null

Fail to reject null

Reject the null hypothesis. Meaning, There is an association between "video" and language. They are not independent.

Fail to reject null

Fail to reject null

Fail to reject null

Fail to reject null

Fail to reject null

Reject the null hypothesis. Meaning, There is an association between "using" and language. They are not independent.

Fail to reject null

Fail to reject null

Fail to reject null

Fail to reject null

Fail to reject null

Fail to reject null

Fail to reject null

Fail to reject null

Fail to reject null

Reject the null hypothesis. Meaning, There is an association between "free" and language. They are not independent.

Fail to reject null

Fail to reject null

Fail to reject null

Fail to reject null



In [557]:
for word in twenty_five:
    observed = pd.crosstab(train[word], train.is_java)
    chi2, p, degf, expected = stats.chi2_contingency(observed)
    alpha = 0.05
    if p < alpha:
        print(f'Reject the null hypothesis. Meaning, There is an association between "{word}" and Java. They are NOT independent.')
    else:
        print(f'Fail to reject null hypothesis. There is not an association between "{word}" and Java. They ARE dependent.')
    print()

Fail to reject null hypothesis. There is not an association between "unknown" and Java. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "apikey" and Java. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "data" and Java. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "api" and Java. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "video" and Java. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "library" and Java. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "const" and Java. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "function" and Java. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "golang" and Java. They ARE dependent.

Fail to reject null hypothesis.

In [558]:
for word in twenty_five:
    observed = pd.crosstab(train[word], train.is_python)
    chi2, p, degf, expected = stats.chi2_contingency(observed)
    alpha = 0.05
    if p < alpha:
        print(f'Reject the null hypothesis. Meaning, There is an association between "{word}" and Python. They are not independent.')
    else:
        print(f'Fail to reject null hypothesis. There is not an association between "{word}" and Python. They ARE dependent.')
    print()

Fail to reject null hypothesis. There is not an association between "unknown" and Python. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "apikey" and Python. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "data" and Python. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "api" and Python. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "video" and Python. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "library" and Python. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "const" and Python. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "function" and Python. They ARE dependent.

Fail to reject null hypothesis. There is not an association between "golang" and Python. They ARE dependent.

Fail to rejec

In [559]:
for word in twenty_five:
    observed = pd.crosstab(train[word], train.is_typescript)
    chi2, p, degf, expected = stats.chi2_contingency(observed)
    alpha = 0.05
    if p < alpha:
        print(f'Reject the null hypothesis. Meaning, There is an association between "{word}" and Typescript. They are not independent.')
    else:
        print(f'Fail to reject null hypothesis. There is not an association between "{word}" and Typescript. They ARE independent.')
    print()

Fail to reject null hypothesis. There is not an association between "unknown" and Typescript. They ARE independent.

Fail to reject null hypothesis. There is not an association between "apikey" and Typescript. They ARE independent.

Fail to reject null hypothesis. There is not an association between "data" and Typescript. They ARE independent.

Fail to reject null hypothesis. There is not an association between "api" and Typescript. They ARE independent.

Fail to reject null hypothesis. There is not an association between "video" and Typescript. They ARE independent.

Fail to reject null hypothesis. There is not an association between "library" and Typescript. They ARE independent.

Fail to reject null hypothesis. There is not an association between "const" and Typescript. They ARE independent.

Fail to reject null hypothesis. There is not an association between "function" and Typescript. They ARE independent.

Fail to reject null hypothesis. There is not an association between "golang

In [560]:
for word in twenty_five:
    observed = pd.crosstab(train[word], train.is_c)
    chi2, p, degf, expected = stats.chi2_contingency(observed)
    alpha = 0.05
    if p < alpha:
        print(f'Reject the null hypothesis. Meaning, There is an association between "{word}" and C. They are not independent.')
    else:
        print(f'Fail to reject null hypothesis. There is not an association between "{word}" and C. They ARE independent.')
    print()

Fail to reject null hypothesis. There is not an association between "unknown" and C. They ARE independent.

Fail to reject null hypothesis. There is not an association between "apikey" and C. They ARE independent.

Fail to reject null hypothesis. There is not an association between "data" and C. They ARE independent.

Fail to reject null hypothesis. There is not an association between "api" and C. They ARE independent.

Fail to reject null hypothesis. There is not an association between "video" and C. They ARE independent.

Fail to reject null hypothesis. There is not an association between "library" and C. They ARE independent.

Fail to reject null hypothesis. There is not an association between "const" and C. They ARE independent.

Fail to reject null hypothesis. There is not an association between "function" and C. They ARE independent.

Fail to reject null hypothesis. There is not an association between "golang" and C. They ARE independent.

Fail to reject null hypothesis. There is

In [561]:
for word in twenty_five:
    observed = pd.crosstab(train[word], train.is_other)
    chi2, p, degf, expected = stats.chi2_contingency(observed)
    alpha = 0.05
    if p < alpha:
        print(f'Reject the null hypothesis. Meaning, There is an association between "{word}" and other languages. They are not independent.')
    else:
        print(f'Fail to reject null hypothesis. There is not an association between "{word}" and other languages. They ARE independent.')
    print()

Fail to reject null hypothesis. There is not an association between "unknown" and other languages. They ARE independent.

Fail to reject null hypothesis. There is not an association between "apikey" and other languages. They ARE independent.

Fail to reject null hypothesis. There is not an association between "data" and other languages. They ARE independent.

Fail to reject null hypothesis. There is not an association between "api" and other languages. They ARE independent.

Fail to reject null hypothesis. There is not an association between "video" and other languages. They ARE independent.

Fail to reject null hypothesis. There is not an association between "library" and other languages. They ARE independent.

Fail to reject null hypothesis. There is not an association between "const" and other languages. They ARE independent.

Fail to reject null hypothesis. There is not an association between "function" and other languages. They ARE independent.

Fail to reject null hypothesis. The

In [565]:
twenty_five

Index(['unknown', 'apikey', 'data', 'api', 'video', 'library', 'const',
       'function', 'golang', 'use', 'using', 'error', 'read', 'use', 'tldr',
       'vuejs', 'component', 'act', 'library', 'simple', 'free', 'source',
       'data', 'tool', 'web'],
      dtype='object')

In [562]:
validate.head()

,title,readme,lemmatized,language,unknown,apikey,data,api,video,library,...,simple,free,source,tool,web,is_java,is_python,is_typescript,is_c,is_other
82,frp,gin web framework getting started prerequisite...,"[web, framework, getting, started, prerequisit...",other,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
26,computer-science,python - 100天从新手到大师 python应用领域和职业发展分析 day01~15...,"[100, day0115, day01, day02, day03, day04, day...",python,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
67,thefuck,chapters past presentations info other good pl...,"[chapter, past, presentation, info, good, plac...",other,0,0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,1
20,linux,documentation terms of service about flutter b...,"[documentation, term, service, beautiful, user...",other,0,0,0,0,1,1,...,0,1,1,1,1,0,0,0,0,1
54,scrcpy,o features install getting started additional ...,"[feature, install, getting, started, additiona...",c,0,0,0,1,0,0,...,1,0,1,0,1,0,0,0,1,0


In [571]:
len(train.columns)

31

In [ ]:
java_x = ['library', 'const','function', 'golang', 'use']

In [572]:
# For java
X_train = train.drop(columns=['title', 'readme', 'lemmatized', 'language', 'is_java', 'is_python', 'is_typescript', 'is_c', 'is_other', 'free', 'unknown' , 
                            'apikey', 'data', 'api', 'video', 'tool', 'vuejs', 'component', 'simple', 'source', 'tool', 'video', 'using', 'read', 'tldr', 'act', 'web'])
y_train = train.is_java

X_validate = validate.drop(columns=['title', 'readme', 'lemmatized', 'language', 'is_java', 'is_python', 'is_typescript', 'is_c', 'is_other', 'free', 'unknown' , 
                            'apikey', 'data', 'api', 'video', 'tool', 'vuejs', 'component', 'simple', 'source', 'tool', 'video', 'using', 'read', 'tldr', 'act', 'web'])
y_validate = validate.is_java

In [574]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import\
DecisionTreeClassifier as DT

In [575]:
# KNN on train java
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88        44
           1       0.00      0.00      0.00        12

    accuracy                           0.79        56
   macro avg       0.39      0.50      0.44        56
weighted avg       0.62      0.79      0.69        56



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [577]:
# KNN on validate java
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(X_validate, y_validate)
y_pred = knn.predict(X_validate)
print(classification_report(y_validate, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.95      0.92        19
           1       0.75      0.60      0.67         5

    accuracy                           0.88        24
   macro avg       0.82      0.77      0.79        24
weighted avg       0.87      0.88      0.87        24



In [578]:
# Logreg on train java
logit = LogisticRegression(C=1, random_state=123)
logit.fit(X_train, y_train)
y_pred = logit.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88        44
           1       0.00      0.00      0.00        12

    accuracy                           0.79        56
   macro avg       0.39      0.50      0.44        56
weighted avg       0.62      0.79      0.69        56



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [579]:
# Logreg on validate java
logit = LogisticRegression(C=1, random_state=123)
logit.fit(X_validate, y_validate)
y_pred = logit.predict(X_validate)
print(classification_report(y_validate, y_pred))

              precision    recall  f1-score   support

           0       0.83      1.00      0.90        19
           1       1.00      0.20      0.33         5

    accuracy                           0.83        24
   macro avg       0.91      0.60      0.62        24
weighted avg       0.86      0.83      0.79        24



In [580]:
# Dectree on train java
clf = DT(max_depth=2)
clf.fit(X_train, y_train)
model_1_preds = clf.predict(X_train)
print(classification_report(y_train, model_1_preds))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88        44
           1       0.00      0.00      0.00        12

    accuracy                           0.79        56
   macro avg       0.39      0.50      0.44        56
weighted avg       0.62      0.79      0.69        56



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [581]:
# Dectree on validate java
clf = DT(max_depth=2)
clf.fit(X_validate, y_validate)
model_1_preds = clf.predict(X_validate)
print(classification_report(y_validate, model_1_preds))

              precision    recall  f1-score   support

           0       0.83      1.00      0.90        19
           1       1.00      0.20      0.33         5

    accuracy                           0.83        24
   macro avg       0.91      0.60      0.62        24
weighted avg       0.86      0.83      0.79        24



In [583]:
five_python

Index(['unknown', 'apikey', 'data', 'api', 'video'], dtype='object')

In [584]:
# For python
X_train = train.drop(columns=['library', 'const', 'function', 'golang', 'use', 'title', 'readme', 'lemmatized', 'language', 'is_java', 'is_python', 'is_typescript', 'is_c', 'is_other', 'free',  'tool', 'vuejs', 'component', 'simple', 'source', 'tool', 'video', 'using', 'read', 'tldr', 'act', 'web'])
y_train = train.is_python

X_validate = validate.drop(columns=['library', 'const', 'function', 'golang', 'use', 'title', 'readme', 'lemmatized', 'language', 'is_java', 'is_python', 'is_typescript', 'is_c', 'is_other', 'free', 'tool', 'vuejs', 'component', 'simple', 'source', 'tool', 'video', 'using', 'read', 'tldr', 'act', 'web'])

y_validate = validate.is_python

In [585]:
# KNN on train python
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95        51
           1       0.00      0.00      0.00         5

    accuracy                           0.91        56
   macro avg       0.46      0.50      0.48        56
weighted avg       0.83      0.91      0.87        56



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [586]:
# KNN on validate python
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(X_validate, y_validate)
y_pred = knn.predict(X_validate)
print(classification_report(y_validate, y_pred))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86        18
           1       0.00      0.00      0.00         6

    accuracy                           0.75        24
   macro avg       0.38      0.50      0.43        24
weighted avg       0.56      0.75      0.64        24



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [587]:
# Logreg on train python
logit = LogisticRegression(C=1, random_state=123)
logit.fit(X_train, y_train)
y_pred = logit.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95        51
           1       0.00      0.00      0.00         5

    accuracy                           0.91        56
   macro avg       0.46      0.50      0.48        56
weighted avg       0.83      0.91      0.87        56



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [588]:
# Logreg on validate python
logit = LogisticRegression(C=1, random_state=123)
logit.fit(X_validate, y_validate)
y_pred = logit.predict(X_validate)
print(classification_report(y_validate, y_pred))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86        18
           1       0.00      0.00      0.00         6

    accuracy                           0.75        24
   macro avg       0.38      0.50      0.43        24
weighted avg       0.56      0.75      0.64        24



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [589]:
# Dectree on train python
clf = DT(max_depth=2)
clf.fit(X_train, y_train)
model_1_preds = clf.predict(X_train)
print(classification_report(y_train, model_1_preds))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96        51
           1       1.00      0.20      0.33         5

    accuracy                           0.93        56
   macro avg       0.96      0.60      0.65        56
weighted avg       0.93      0.93      0.91        56



In [590]:
# Dectree on validate python
clf = DT(max_depth=2)
clf.fit(X_validate, y_validate)
model_1_preds = clf.predict(X_validate)
print(classification_report(y_validate, model_1_preds))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86        18
           1       0.00      0.00      0.00         6

    accuracy                           0.75        24
   macro avg       0.38      0.50      0.43        24
weighted avg       0.56      0.75      0.64        24



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [592]:
five_python

Index(['unknown', 'apikey', 'data', 'api', 'video'], dtype='object')

In [591]:
five_typescript

Index(['using', 'error', 'read', 'use', 'tldr'], dtype='object')

In [595]:
# For typescript
X_train = train.drop(columns=['unknown', 'apikey', 'data', 'api', 'video', 'title', 'readme', 'lemmatized', 'language', 'is_java', 'is_python', 'is_typescript', 'is_c', 'is_other', 'free', 'unknown' , 
                            'apikey', 'data', 'api', 'video', 'library', 'golang', 'simple', 'tool', 'vuejs', 'component', 'simple', 'source', 'tool', 'video', 'web'])
y_train = train.is_typescript

X_validate = validate.drop(columns=['unknown', 'apikey', 'data', 'api', 'video', 'title', 'readme', 'lemmatized', 'language', 'is_java', 'is_python', 'is_typescript', 'is_c', 'is_other', 'free', 'unknown' , 
                            'apikey', 'data', 'api', 'video', 'library', 'golang', 'simple', 'tool', 'vuejs', 'component', 'simple', 'source', 'tool', 'video', 'web'])

y_validate = validate.is_typescript

In [596]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        49
           1       0.00      0.00      0.00         7

    accuracy                           0.88        56
   macro avg       0.44      0.50      0.47        56
weighted avg       0.77      0.88      0.82        56



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [597]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(X_validate, y_validate)
y_pred = knn.predict(X_validate)
print(classification_report(y_validate, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        21
           1       0.00      0.00      0.00         3

    accuracy                           0.88        24
   macro avg       0.44      0.50      0.47        24
weighted avg       0.77      0.88      0.82        24



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [598]:
logit = LogisticRegression(C=1, random_state=123)
logit.fit(X_train, y_train)
y_pred = logit.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        49
           1       0.00      0.00      0.00         7

    accuracy                           0.88        56
   macro avg       0.44      0.50      0.47        56
weighted avg       0.77      0.88      0.82        56



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [599]:
logit = LogisticRegression(C=1, random_state=123)
logit.fit(X_validate, y_validate)
y_pred = logit.predict(X_validate)
print(classification_report(y_validate, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        21
           1       0.00      0.00      0.00         3

    accuracy                           0.88        24
   macro avg       0.44      0.50      0.47        24
weighted avg       0.77      0.88      0.82        24



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [600]:
clf = DT(max_depth=2)
clf.fit(X_train, y_train)
model_1_preds = clf.predict(X_train)
print(classification_report(y_train, model_1_preds))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        49
           1       0.00      0.00      0.00         7

    accuracy                           0.88        56
   macro avg       0.44      0.50      0.47        56
weighted avg       0.77      0.88      0.82        56



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [601]:
clf = DT(max_depth=2)
clf.fit(X_validate, y_validate)
model_1_preds = clf.predict(X_validate)
print(classification_report(y_validate, model_1_preds))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        21
           1       0.00      0.00      0.00         3

    accuracy                           0.88        24
   macro avg       0.44      0.50      0.47        24
weighted avg       0.77      0.88      0.82        24



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [604]:
five_typescript

Index(['using', 'error', 'read', 'use', 'tldr'], dtype='object')

In [603]:
five_c

Index(['vuejs', 'component', 'act', 'library', 'simple'], dtype='object')

In [ ]:
'package', 'roadmap', 'build', 'xpy', 'install', 'license'

In [606]:
# For c
X_train = train.drop(columns=['using', 'error', 'read', 'use', 'tldr', 'title', 'readme', 'lemmatized', 'language', 'is_java', 'is_python', 'is_typescript', 'is_c', 'is_other', 'free', 'unknown' , 
                            'apikey', 'data', 'api', 'video', 'golang', 'tool', 'source', 'tool', 'video', 'web'])
y_train = train.is_c

X_validate = validate.drop(columns=['using', 'error', 'read', 'use', 'tldr', 'title', 'readme', 'lemmatized', 'language', 'is_java', 'is_python', 'is_typescript', 'is_c', 'is_other', 'free', 'unknown' , 
                            'apikey', 'data', 'api', 'video', 'golang', 'tool', 'source', 'tool', 'video', 'web'])

y_validate = validate.is_c

In [607]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96        52
           1       0.00      0.00      0.00         4

    accuracy                           0.93        56
   macro avg       0.46      0.50      0.48        56
weighted avg       0.86      0.93      0.89        56



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [608]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(X_validate, y_validate)
y_pred = knn.predict(X_validate)
print(classification_report(y_validate, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        21
           1       0.00      0.00      0.00         3

    accuracy                           0.88        24
   macro avg       0.44      0.50      0.47        24
weighted avg       0.77      0.88      0.82        24



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [609]:
logit = LogisticRegression(C=1, random_state=123)
logit.fit(X_train, y_train)
y_pred = logit.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96        52
           1       0.00      0.00      0.00         4

    accuracy                           0.93        56
   macro avg       0.46      0.50      0.48        56
weighted avg       0.86      0.93      0.89        56



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [610]:
logit = LogisticRegression(C=1, random_state=123)
logit.fit(X_validate, y_validate)
y_pred = logit.predict(X_validate)
print(classification_report(y_validate, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        21
           1       0.00      0.00      0.00         3

    accuracy                           0.88        24
   macro avg       0.44      0.50      0.47        24
weighted avg       0.77      0.88      0.82        24



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [611]:
clf = DT(max_depth=2)
clf.fit(X_train, y_train)
model_1_preds = clf.predict(X_train)
print(classification_report(y_train, model_1_preds))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96        52
           1       0.00      0.00      0.00         4

    accuracy                           0.93        56
   macro avg       0.46      0.50      0.48        56
weighted avg       0.86      0.93      0.89        56



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [612]:
clf = DT(max_depth=2)
clf.fit(X_validate, y_validate)
model_1_preds = clf.predict(X_validate)
print(classification_report(y_validate, model_1_preds))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        21
           1       0.00      0.00      0.00         3

    accuracy                           0.88        24
   macro avg       0.44      0.50      0.47        24
weighted avg       0.77      0.88      0.82        24



/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Bigrams

In [ ]:
five_python_bigrams = pd.Series(python_bigrams).value_counts().head().index
five_java_bigrams = pd.Series(java_bigrams).value_counts().head().index
five_typescript_bigrams = pd.Series(typescript_bigrams).value_counts().head().index
five_c_bigrams = pd.Series(c_bigrams).value_counts().head().index
five_other_bigrams = pd.Series(other_bigrams).value_counts().head().index

five_python_trigrams = pd.Series(python_trigrams).value_counts().head().index
five_java_trigrams = pd.Series(java_trigrams).value_counts().head().index
five_typescript_trigrams = pd.Series(typescript_trigrams).value_counts().head().index
five_c_trigrams = pd.Series(c_trigrams).value_counts().head().index
five_other_trigrams = pd.Series(other_trigrams).value_counts().head().index

twenty_five_bigrams = five_python_bigrams.append(five_java_bigrams)
twenty_five_bigrams = twenty_five_bigrams.append(five_typescript_bigrams)
twenty_five_bigrams = twenty_five_bigrams.append(five_c_bigrams)
twenty_five_bigrams = twenty_five_bigrams.append(five_other_bigrams)
twenty_five_bigrams = [' '.join(x) for x in twenty_five_bigrams]

twenty_five_trigrams = five_python_trigrams.append(five_java_trigrams)
twenty_five_trigrams = twenty_five_trigrams.append(five_typescript_trigrams)
twenty_five_trigrams = twenty_five_trigrams.append(five_c_trigrams)
twenty_five_trigrams = twenty_five_trigrams.append(five_other_trigrams)
twenty_five_trigrams = [' '.join(x) for x in twenty_five_trigrams]

In [ ]:
for i in range(len(twenty_five_bigrams)):
    train[f'{twenty_five_bigrams[i]}'] = train['lemmatized'].apply(lambda x: 1 if twenty_five_bigrams[i] in x else 0)

In [ ]:
train.(component vuejs)

In [ ]:
train.head()

In [ ]:
for word in twenty_five_bigrams:
    observed = pd.crosstab(train[word], train.is_other)
    chi2, p, degf, expected = stats.chi2_contingency(observed)
    alpha = 0.05
    if p < alpha:
        print(f'Reject the null hypothesis. Meaning, There is an association between "{word}" and other languages. They are not independent.')
    else:
        print(f'Fail to reject null hypothesis. There is not an association between "{word}" and other languages. They ARE independent.')
    print()

In [ ]:
for i in range(len(twenty_five_trigrams)):
    train[f'{twenty_five_trigrams[i]}'] = train['lemmatized'].apply(lambda x: 1 if twenty_five_trigrams[i] in x else 0)

In [ ]:
for word in twenty_five_trigrams:
    observed = pd.crosstab(train[word], train.is_other)
    chi2, p, degf, expected = stats.chi2_contingency(observed)
    alpha = 0.05
    if p < alpha:
        print(f'Reject the null hypothesis. Meaning, There is an association between "{word}" and other languages. They are not independent.')
    else:
        print(f'Fail to reject null hypothesis. There is not an association between "{word}" and other languages. They ARE independent.')
    print()

In [ ]:
train.head()